In [1]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [1]:
# import xgboost as xgb
# from xgboost.sklearn import XGBClassifier

import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, History 

from sklearn import metrics
from sklearn.metrics import roc_auc_score

# import matplotlib.pyplot as plt

from sklearn.externals import joblib

from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
import pickle

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np


# fix random seed for reproducibility
np.random.seed(7)

import psutil

import time

### Model trainig with hyper-parameter tuning

In [3]:
def create_model(params): 
    
    x_train_temp = train_sample[df_features].copy() 
    y_train_temp = train_sample[df_target].copy()
    
    model = Sequential()
    model.add(Dense(params['units1'], input_shape=(x_train_temp.shape[1],)))
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout1']))
    if(params['num_layers'] == 'two_hidden'):
        model.add(Dense(params['units2']))
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dropout2']))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', metrics=['accuracy'],
                  optimizer=params['optimizer'])
    
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=params['early_stop_rounds'])
    history = History()
    
    model.fit(x_train_temp, y_train_temp,
              batch_size=params['batch_size'],
              epochs=1,
              callbacks=[early_stop, history],
              verbose=2,
              validation_data=(valid_sample[df_features],valid_sample[df_target])) 
    
    [loss, acc] = model.evaluate(valid_sample[df_features],valid_sample[df_target], verbose=0)
    
    global num
    mem = psutil.virtual_memory()
    
    if(np.isnan(acc)):
        
        print("{}) Validation set Accuracy: NaN".format(num),"\tAvailable Mem:",(mem.available/1024)/1024,"mb")
        
        
        f1 = open("status.txt", "a+")
        f1.write("num:{} , loss:{}, Valid acc: {}, params: {} \n \n".format(num,loss, acc,params))
        f1.close() 
      
        
        num = num + 1
        return {'loss': np.inf, 'status': STATUS_OK, 'paras': params}
    else:
        
        model.fit(x_train_temp, y_train_temp,
                  batch_size=params['batch_size'],
                  epochs=5,
                  callbacks=[early_stop, history],
                  verbose=2,
                  validation_data=(valid_sample[df_features],valid_sample[df_target]))
        
        [loss, acc] = model.evaluate(valid_sample[df_features],valid_sample[df_target], verbose=0)
        
        mem = psutil.virtual_memory()
        if(np.isnan(acc)):
            print("{}) Validation set Accuracy: NaN".format(num),"\tAvailable Mem:",(mem.available/1024)/1024,"mb")
            
            
            f1 = open("status.txt", "a+")
            f1.write("num:{} , loss:{}, Valid acc: {}, params: {} \n \n".format(num,loss, acc,params))
            f1.close() 
            
            num = num + 1
            return {'loss': np.inf, 'status': STATUS_OK, 'params': params}
        print("{}) Validation set Accuracy: {:7.2f}%".format(num,acc*100),"\tAvailable Mem:",(mem.available/1024)/1024,"mb")
        
        f1 = open("status.txt", "a+")
        f1.write("num:{} , loss:{}, Valid acc: {}, params: {} \n \n".format(num,loss, acc,params))
        f1.close() 
        
        num = num + 1
        return {'loss': -acc, 'status': STATUS_OK, 'params': params}
    
    
def get_best_model_nn(space):

    trials=Trials()
    global num
    num = 1
    best = fmin(create_model, space, algo=tpe.suggest, max_evals=5, trials=trials)

    return trials.best_trial['result']

def train_best_model(best_params):

    x_train_temp = train_sample[df_features].copy() 
    y_train_temp = train_sample[df_target].copy()
    model = Sequential()
    model.add(Dense(best_params['units1'], input_shape=(x_train_temp.shape[1],)))
    model.add(Activation(best_params['activation']))
    model.add(Dropout(best_params['dropout1']))
    if(best_params['num_layers'] == 'two_hidden'):
        model.add(Dense(best_params['units2']))
        model.add(Activation(best_params['activation']))
        model.add(Dropout(best_params['dropout2']))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', metrics=['accuracy'],
                  optimizer=best_params['optimizer'])

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=best_params['early_stop_rounds'])
    history = History()

    model.fit(x_train_temp, y_train_temp,
              batch_size=best_params['batch_size'],
              epochs=500,
              callbacks=[early_stop, history],
              verbose=2,
              validation_data=(valid_sample[df_features],valid_sample[df_target]))
    return model

In [4]:
train_o = pd.read_pickle('../../data/train_insurance_2017.pkl')
valid_o = pd.read_pickle('../../data/valid_insurance_2017.pkl')
# test_o = pd.read_pickle('../../data/test_insurance_2017.pkl')

In [5]:
# train_o.drop(['StartedFlag'],axis=1,inplace=True)
# valid_o.drop(['StartedFlag'],axis=1,inplace=True)
# # test_o.drop(['StartedFlag'],axis=1,inplace=True)

In [6]:
# train, valid, test, df_features, df_target = data(all_data,target='NewEstimateTotal')
# del all_data
df_target = 'NewEstimateTotal'
# df_features = list(set(train_o.columns) - set([df_target]))
df_features = list(train_o.columns)

In [7]:
df_features.remove('NewEstimateTotal')
df_features.remove('StartedFlag')

In [8]:
import pickle
with open('df_features.pkl', 'wb') as f:
     pickle.dump(df_features, f)   

In [9]:
train_sample = train_o.head(100)
valid_sample = valid_o.head(20)
# del train
# del valid
del train_o
del valid_o
# del test_o
# del test

In [10]:
start_time = time.time()
space = {'num_layers': hp.choice('num_layers',['one_hidden', 'two_hidden']),
                'units1': hp.choice('units1', [32, 64, 128, 256,512]),
                'units2': hp.choice('units2', [32, 64, 128, 256,512]),
                'dropout1': hp.uniform('dropout1', .25,.75),
                'dropout2': hp.uniform('dropout2',  .25,.75),
                'batch_size' : hp.choice('batch_size', [16,32,64,128]),
                'nb_epochs' :  500,
                'optimizer': hp.choice('optimizer',['rmsprop', 'adam', 'nadam','sgd']),
                'activation': hp.choice('activation',['relu','sigmoid']),
                'early_stop_rounds': hp.choice('early_stop_rounds',[10,20,30,40,50]),
            }
best_trails1 = get_best_model_nn(space)


f = open("estimate_total_100000_hyperopt_keras_2017.txt", "w")
f.write("Took {} seconds with best trails as: {}".format(time.time()-start_time,best_trails1))
f.close() 

# Training with the best params

model = train_best_model(best_trails1['params'])

# To save model
# best_model.save('insurance_classify_100000_estimate_total_hyperopt_keras_2017.h5')

# serialize model to JSON
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.h5")
print("Saved model to disk")

# with open('insurance_classify_100000_estimate_total_hyperopt_2017.pkl', 'wb') as f:
#     pickle.dump(best_model, f)

Train on 100 samples, validate on 20 samples
Epoch 1/1
 - 1s - loss: 0.7124 - acc: 0.5800 - val_loss: 0.6054 - val_acc: 0.6500
Train on 100 samples, validate on 20 samples
Epoch 1/5
 - 0s - loss: 0.6563 - acc: 0.6400 - val_loss: 0.5741 - val_acc: 0.7500
Epoch 2/5
 - 0s - loss: 0.5697 - acc: 0.7400 - val_loss: 0.5738 - val_acc: 0.7000
Epoch 3/5
 - 0s - loss: 0.5398 - acc: 0.7300 - val_loss: 0.5678 - val_acc: 0.7000
Epoch 4/5
 - 0s - loss: 0.5520 - acc: 0.6400 - val_loss: 0.5771 - val_acc: 0.6500
Epoch 5/5
 - 0s - loss: 0.5070 - acc: 0.7900 - val_loss: 0.5844 - val_acc: 0.6500
1) Validation set Accuracy:   65.00% 	Available Mem: 29918.7578125 mb
Train on 100 samples, validate on 20 samples
Epoch 1/1
 - 0s - loss: 0.8577 - acc: 0.4200 - val_loss: 0.6393 - val_acc: 0.6500
Train on 100 samples, validate on 20 samples
Epoch 1/5
 - 0s - loss: 0.7800 - acc: 0.5200 - val_loss: 0.6363 - val_acc: 0.6500
Epoch 2/5
 - 0s - loss: 0.7469 - acc: 0.5300 - val_loss: 0.6299 - val_acc: 0.6500
Epoch 3/5
 -

In [29]:
# To load the pickle model\n"
# from keras.models import load_model
# load_model('insurance_classify_100000_estimate_total_hyperopt_keras_2017.h5')

In [17]:
# valid.loc[:,'prob_up'] = model.predict_proba(valid[df_features])[:,1]

# valid['prediction'] = np.where(valid.prob_up > 0.5,1,0)

# accuracy_score(valid.NewEstimateTotal, valid.prediction)

In [18]:
# test.loc[:,'prob_up'] = model.predict_proba(test[df_features])[:,1]

# test['prediction'] = np.where(test.prob_up > 0.5,1,0)

# accuracy_score(test.NewEstimateTotal, test.prediction)

In [ ]:
# Making predictions on the Test data

In [9]:
from keras.models import load_model
best_model = load_model('insurance_classify_100000_estimate_total_hyperopt_keras_2017.h5')

Using TensorFlow backend.


In [11]:
test_o = pd.read_pickle('../../data/test_insurance_2017.pkl')
# test_o.drop(['StartedFlag'],axis=1,inplace=True)

In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [12]:
test_new = pd.DataFrame()
for i,chunk in enumerate(np.array_split(test_o, 5)):
    print(i)
    pred = model.predict(chunk[df_features]).flatten()
    chunk['preds'] = np.where(pred>=0.5,1,0)
#     print(type(chunk))
#     print(chunk.head(5))
#     break
    test_new = test_new.append(chunk)
#     print(type(test_new))
# test_new.to_pickle('test_insurance_2017_preds_estimate_total_hyperopt.pkl')
acc = accuracy_score(test_new.NewEstimateTotal, test_new.preds)

0
1
2
3
4


In [13]:
acc

0.7360813639864876

In [1]:
# # Confusion Matrix on Test Data
# metrics.confusion_matrix(test_new.NewEstimateTotal, test_new.preds)

In [ ]:
# Saving test csv result

In [8]:
test_new = pd.read_pickle('test_insurance_2017_preds_estimate_total_hyperopt_keras.pkl')

In [9]:
def reverse_dummies(dummies_df):
    sep = '_'
    dummy_cols = list(set(col.split(sep)[0] for col in dummies_df.columns if sep in col))
    print(dummy_cols)
    other_cols = [col for col in dummies_df.columns if sep not in col]
    print(other_cols)
    dfs = []
    for i, col in enumerate(dummy_cols):
        print(i, col)
        dfs.append(dummies_df.filter(regex=col).rename(columns=lambda name: name.split(sep)[1]).idxmax(axis=1))

    df = pd.concat(dfs + [dummies_df[other_cols]], axis=1)
    df.columns = dummy_cols + other_cols
    return df

In [10]:
test_result = reverse_dummies(test_new)

['DisplayName', 'State', 'DivisionName', 'CommOrRes']
['NoteCount', 'PhotoCount', 'JobCount', 'ClaimCount', 'LossYearMo', 'NewEstimateTotal', 'preds']
0 DisplayName
1 State
2 DivisionName
3 CommOrRes


In [12]:
test_result.to_csv('test_insurance_2017_preds_estimate_total_hyperopt_keras.csv')